# Convolution neural network

## Set Packages

In [1]:
import tensorflow
import pandas as pd, numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio # settings like template by default

pio.templates.default = 'plotly_dark' # default template

from tensorflow.keras.models import Sequential, Model 
from tensorflow.keras.layers import Dense, Flatten, Input, LSTM
from tensorflow.keras.layers import Conv1D, MaxPooling1D

gpus = tensorflow.config.list_physical_devices('GPU')  # obtiene las gpus instaladas
if len(gpus) > 0:   # si hay alguna gpu instalada
    # setea para alocar apenas a memória da GPU necessária para as alocações de tempo de execução
    tf.config.experimental.set_memory_growth(gpus[0], True) 
print(gpus)

[]


In [86]:
# Read data
df = pd.read_csv('data/airline-passengers.csv', header=0, index_col=0, parse_dates=True)
print(f"Size data: {df.shape}")
print(f"min date: {min(df.index)} and max date: {max(df.index)}")
df.head(3)

Size data: (144, 1)
min date: 1949-01-01 00:00:00 and max date: 1960-12-01 00:00:00


,Passengers
Month,
1949-01-01,112
1949-02-01,118
1949-03-01,132


In [9]:
train = df.loc[:'1959-12-01']
test = df.loc['1959-01-01':] # test set starts before the end of the train set
print(f"Train set: {train.shape} and Test set: {test.shape}")
print(f"Train maz date: {max(train.index)} and test min date {min(test.index)}")

Train set: (132, 1) and Test set: (24, 1)
Train maz date: 1959-12-01 00:00:00 and test min date 1959-01-01 00:00:00


In [10]:
raw_seq = train.values.flatten()
print(f"Raw sequence length: {len(raw_seq)}")
print(raw_seq)

Raw sequence length: 132
[112 118 132 129 121 135 148 148 136 119 104 118 115 126 141 135 125 149
 170 170 158 133 114 140 145 150 178 163 172 178 199 199 184 162 146 166
 171 180 193 181 183 218 230 242 209 191 172 194 196 196 236 235 229 243
 264 272 237 211 180 201 204 188 235 227 234 264 302 293 259 229 203 229
 242 233 267 269 270 315 364 347 312 274 237 278 284 277 317 313 318 374
 413 405 355 306 271 306 315 301 356 348 355 422 465 467 404 347 305 336
 340 318 362 348 363 435 491 505 404 359 310 337 360 342 406 396 420 472
 548 559 463 407 362 405]


## Univariate series, one output (One step)

In [19]:
# rolling window from univariate time series
def split_sequence(sequence: list, n_steps: int, n_features: int):
    X, y = list(), list()
    for i in range(len(sequence)):
        # calculate sequence end index
        end_ix = i + n_steps
        # if index is greater than the length of the sequence, stop generating samples
        if end_ix > len(sequence)-1:
            break
        
        X.append( sequence[i:end_ix] ) # add the current sequence slice to the input features
        y.append( sequence[end_ix] ) # add the next value as the target variable
    
    X = np.array(X)
    # reshape from [samples, timesteps] to [samples, timesteps, features]
    X = X.reshape((X.shape[0], X.shape[1], n_features))
    y = np.array(y)
    return X, y

# number of steps
n_steps = 12
n_features = 1

# Generate train sample
X, y = split_sequence(raw_seq, n_steps, n_features)
print(X.shape, y.shape)

# Show the first sample
for i in range(1):
    print(X[i].tolist(), y[i])

(120, 12, 1) (120,)
[[112], [118], [132], [129], [121], [135], [148], [148], [136], [119], [104], [118]] 115


In [20]:
# define una modelo de red neuronal CNN
model1 = Sequential()
model1.add(Input(shape=(n_steps, n_features)))
model1.add(Conv1D(filters=64, kernel_size=2, activation='relu')) #64 filtros de tamaño 2
model1.add(MaxPooling1D(pool_size=2)) # maxpooling size = 2
model1.add(Flatten())  # flaten layer
model1.add(Dense(50, activation='relu'))  # dense layer
model1.add(Dense(1))  # one output neuron
model1.compile(optimizer='adam', loss='mse')
# Train data
model1.fit(X, y, epochs=100, verbose=False)
# modelo.summary()

In [22]:
losses_df1 = pd.DataFrame(model1.history.history)
fig = px.line(losses_df1, x = losses_df1.index, y='loss',
            title='Loss in CNN model with one step ')
fig.show()

In [23]:
# Generate test sample
X_test, y_test = split_sequence(test.values.flatten(), n_steps, n_features)
print(X_test.shape, y_test.shape)
for i, val in enumerate(X_test):
    print(f"Test sample {i+1}: {val.tolist()} -> Target: {y_test[i]}")

(12, 12, 1) (12,)
Test sample 1: [[360], [342], [406], [396], [420], [472], [548], [559], [463], [407], [362], [405]] -> Target: 417
Test sample 2: [[342], [406], [396], [420], [472], [548], [559], [463], [407], [362], [405], [417]] -> Target: 391
Test sample 3: [[406], [396], [420], [472], [548], [559], [463], [407], [362], [405], [417], [391]] -> Target: 419
Test sample 4: [[396], [420], [472], [548], [559], [463], [407], [362], [405], [417], [391], [419]] -> Target: 461
Test sample 5: [[420], [472], [548], [559], [463], [407], [362], [405], [417], [391], [419], [461]] -> Target: 472
Test sample 6: [[472], [548], [559], [463], [407], [362], [405], [417], [391], [419], [461], [472]] -> Target: 535
Test sample 7: [[548], [559], [463], [407], [362], [405], [417], [391], [419], [461], [472], [535]] -> Target: 622
Test sample 8: [[559], [463], [407], [362], [405], [417], [391], [419], [461], [472], [535], [622]] -> Target: 606
Test sample 9: [[463], [407], [362], [405], [417], [391], [419

In [24]:
updated_array = X_test[0:1, :, :]
list_predicted = []
for index in range(X_test.shape[0]):
    yhat = model1.predict(updated_array, verbose=False)  # predice el punto siguiente
    list_predicted.append(yhat[0, 0].tolist())
    print(f"Predicted value: {yhat.shape} {yhat.tolist()}")
    updated_array = np.concatenate((updated_array[:, 1:, :], yhat.reshape(1, 1, 1)), axis=1)

Predicted value: (1, 1) [[396.3074645996094]]
Predicted value: (1, 1) [[414.2444152832031]]
Predicted value: (1, 1) [[425.55194091796875]]
Predicted value: (1, 1) [[438.2084045410156]]
Predicted value: (1, 1) [[487.3876953125]]
Predicted value: (1, 1) [[561.6334228515625]]
Predicted value: (1, 1) [[609.091796875]]
Predicted value: (1, 1) [[606.0316162109375]]
Predicted value: (1, 1) [[531.5687866210938]]
Predicted value: (1, 1) [[466.802978515625]]
Predicted value: (1, 1) [[446.43963623046875]]
Predicted value: (1, 1) [[442.4419250488281]]


In [25]:
print(list_predicted)
print(y_test.shape)
print(y_test)
# score = np.sqrt(mean_squared_error(list_predicted, y_test))
rmse = np.sqrt(np.mean((y_test - np.array(list_predicted))**2))
print("RMSE:", rmse)

[396.3074645996094, 414.2444152832031, 425.55194091796875, 438.2084045410156, 487.3876953125, 561.6334228515625, 609.091796875, 606.0316162109375, 531.5687866210938, 466.802978515625, 446.43963623046875, 442.4419250488281]
(12,)
[417 391 419 461 472 535 622 606 508 461 390 432]
RMSE: 23.322450383752088


In [27]:
compared_df = test['1960-01-01':].copy()
compared_df['Predicted'] = list_predicted

In [28]:
fig = px.line(compared_df, x = compared_df.index, y= ['Passengers', 'Predicted'],
            labels={'value': 'Number of passengers', 'variable': 'Legend'},
            title='Prediction with a CNN model (one step)')
fig.show()

## Univariate series, one output (Multi step)

In [31]:
def split_sequence(sequence: list, n_steps_in: int, n_steps_out: int, n_features: int):
    X, y = list(), list()
    for i in range(len(sequence)):
        end_ix = i + n_steps_in 
        out_end_ix = end_ix + n_steps_out
        
        if out_end_ix > len(sequence):
            break
        
        X.append( sequence[i:end_ix] ) 
        y.append( sequence[end_ix:out_end_ix] )
    
    X = np.array(X)
    # reshape from [samples, timesteps] to [samples, timesteps, features]
    X = X.reshape((X.shape[0], X.shape[1], n_features))
    y = np.array(y)
    return X, y

# choose the number of steps for input and output
n_steps_in, n_steps_out, n_features = 12, 3, 1

X, y = split_sequence(raw_seq, n_steps_in, n_steps_out, n_features)
print(X.shape, y.shape)
# show the shape of the generated samples
for i in range(2):
    print(X[i].tolist(), y[i])

(118, 12, 1) (118, 3)
[[112], [118], [132], [129], [121], [135], [148], [148], [136], [119], [104], [118]] [115 126 141]
[[118], [132], [129], [121], [135], [148], [148], [136], [119], [104], [118], [115]] [126 141 135]


In [70]:
model2 = Sequential()
model2.add(Input(shape=(n_steps_in, n_features)))
model2.add(Conv1D(filters=64, kernel_size=2, activation='relu'))
model2.add(MaxPooling1D(pool_size=2))
model2.add(Flatten())
model2.add(Dense(50, activation='relu'))
model2.add(Dense(n_steps_out))
model2.compile(optimizer='adam', loss='mse')
model2.fit(X, y, epochs=100, verbose=0)

In [71]:
losses_df2 = pd.DataFrame(model2.history.history)
fig = px.line(losses_df2, x = losses_df1.index, y='loss', 
            title='Loss in CNN model with multiple steps')
fig.show()

In [79]:
# Generate test sample
X_test, y_test = split_sequence(test.values.flatten(), n_steps_in, n_steps_out, n_features)
print(X_test.shape, y_test.shape)
for i, val in enumerate(X_test):
    print(f"{i+1}: {val.tolist()} -> Target: {y_test[i]}")

(10, 12, 1) (10, 3)
1: [[360], [342], [406], [396], [420], [472], [548], [559], [463], [407], [362], [405]] -> Target: [417 391 419]
2: [[342], [406], [396], [420], [472], [548], [559], [463], [407], [362], [405], [417]] -> Target: [391 419 461]
3: [[406], [396], [420], [472], [548], [559], [463], [407], [362], [405], [417], [391]] -> Target: [419 461 472]
4: [[396], [420], [472], [548], [559], [463], [407], [362], [405], [417], [391], [419]] -> Target: [461 472 535]
5: [[420], [472], [548], [559], [463], [407], [362], [405], [417], [391], [419], [461]] -> Target: [472 535 622]
6: [[472], [548], [559], [463], [407], [362], [405], [417], [391], [419], [461], [472]] -> Target: [535 622 606]
7: [[548], [559], [463], [407], [362], [405], [417], [391], [419], [461], [472], [535]] -> Target: [622 606 508]
8: [[559], [463], [407], [362], [405], [417], [391], [419], [461], [472], [535], [622]] -> Target: [606 508 461]
9: [[463], [407], [362], [405], [417], [391], [419], [461], [472], [535], [6

In [80]:
updated_array = X_test[0:1, :, :]
yhat = model2.predict(updated_array, verbose=False)
yhat

array([[385.64822, 394.83426, 427.26813]], dtype=float32)

In [81]:
updated_array[:, 3:, :].shape, yhat.shape

((1, 9, 1), (1, 3))

In [82]:
updated_array = X_test[0:1, :, :]
list_predicted = []
limit = X_test.shape[0]
for index in range(limit):
    yhat = model2.predict(updated_array, verbose=False)  # predice el punto siguiente
    if index == limit - 1:
        my_end = yhat.flatten().tolist()
        list_predicted.extend(my_end)
        print("end predicted value:", my_end)
    else:
        list_predicted.append(yhat[0, 0].tolist())
    print(f"Predicted value: {yhat.shape} {yhat.tolist()}")
    updated_array = np.concatenate((updated_array[:, 3:, :], yhat.reshape(1, 3, 1)), axis=1)
print(f"length of list_predicted: {len(list_predicted)}")
print(list_predicted)

Predicted value: (1, 3) [[385.6482238769531, 394.8342590332031, 427.26812744140625]]
Predicted value: (1, 3) [[447.2381591796875, 465.622314453125, 541.1231079101562]]
Predicted value: (1, 3) [[563.0306396484375, 569.98388671875, 522.9933471679688]]
Predicted value: (1, 3) [[437.6123352050781, 432.41973876953125, 432.9305419921875]]
Predicted value: (1, 3) [[423.6817321777344, 435.2011413574219, 473.309814453125]]
Predicted value: (1, 3) [[500.69903564453125, 520.9974365234375, 579.0516357421875]]
Predicted value: (1, 3) [[581.198486328125, 588.6232299804688, 555.2052001953125]]
Predicted value: (1, 3) [[472.9999084472656, 471.9551696777344, 468.7039489746094]]
Predicted value: (1, 3) [[465.5152282714844, 481.59674072265625, 523.7171020507812]]
end predicted value: [548.6051635742188, 568.1598510742188, 611.1094970703125]
Predicted value: (1, 3) [[548.6051635742188, 568.1598510742188, 611.1094970703125]]
length of list_predicted: 12
[385.6482238769531, 447.2381591796875, 563.0306396484

In [83]:
compared2_df = test['1960-01-01':].copy()
compared2_df['Predicted'] = list_predicted
compared2_df.head(2)

,Passengers,Predicted
Month,,
1960-01-01,417,385.648224
1960-02-01,391,447.238159


In [84]:
# np.array(list_predicted).shape, y_test[:, 0].shape
rmse = np.sqrt(np.mean((compared2_df.Passengers - compared2_df.Predicted)**2))
print("RMSE:", rmse)

RMSE: 100.66579875068305


In [85]:
fig = px.line(compared2_df, x = compared2_df.index, y= ['Passengers', 'Predicted'],
            labels={'value': 'Number of passengers', 'variable': 'Legend'},
            title='Prediction with a CNN model (multi-step)')
fig.show()